In [1]:
import re   

In [2]:
## Read in the Chinese character pinyin map in a dictionary
Chinese_character_pinyin_map = {}
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/charmap',encoding='utf8') as Chinese_charmap:
    for line in Chinese_charmap:
        Chinese_character_pinyin_map[line[0]] = line[1:].strip()
        
count_yi = 0
for char in Chinese_character_pinyin_map:
    if Chinese_character_pinyin_map[char] == 'yi':
        count_yi+=1
        
print ('The number of possible Chinese characters that are pronounced as yi is {}'.format(count_yi))

The number of possible Chinese characters that are pronounced as yi is 484


In [3]:
## Construct a dict, and the key of the dict is the pinyin in Charmap, and the value is a list contains all the Chinese
## characters matching that pinyin
Chinese_pinyin_character_map = {}
for char in Chinese_character_pinyin_map:
    if Chinese_character_pinyin_map[char] not in Chinese_pinyin_character_map:
        Chinese_pinyin_character_map[Chinese_character_pinyin_map[char]] = [char]
    else:
        Chinese_pinyin_character_map[Chinese_character_pinyin_map[char]].append(char)
        
#print (len(Chinese_pinyin_character_map['yi']))
#print (Chinese_pinyin_character_map['yi'])

In [4]:
## Read in the Chinese training file as a big string
Chinese_train_string = ''
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/train.han',encoding='utf8') as Chinese_train:
    for line in Chinese_train:
        Chinese_train_string += line

print (Chinese_train_string[0:100])        

adam8157_away: 早
MeaCulpa: 早
roylez: 早
好
一直觉得《西游记》有个BUG，唐僧肉吃完长生不死，他为什么不咬自己一口？咬完任你各路妖魔奈我何？轻装上路差旅费省四分之


In [5]:
## Construct unigram word count for the train string
word_count_unigram = {} ## This contains the word counts for unigram in Chinese_train file
for cha in Chinese_train_string:
    if cha not in word_count_unigram:
        word_count_unigram[cha]=1
    else:
        word_count_unigram[cha]+=1

In [6]:
### define function to create the tree structure to store the training parameters
def create(ngram, dic):## ngram is a ngram string, dic is a dictionary. 
    n = len(ngram)
    if n == 0: ##base case 
        return
    if ngram[0] not in dic:
        dic[ngram[0]] = []
        dic[ngram[0]].append(1)
        temp_dict = {}
        dic[ngram[0]].append(temp_dict)
    else:
        dic[ngram[0]][0] += 1
    create(ngram[1:],dic[ngram[0]][1])  

In [47]:
## Train the n-gram model
n = 10  ## 10-gram model
ngram_training = {} ## This is tree structure containing the traing results
                    ## Key is the cha, then value is a list; 1st element in the list is the count of the key,
                    ## 2nd element in the list is a dict in which key is the cha left following the previous key
                    ## for n gram, there are n levels of the tree

for i in range(len(Chinese_train_string)-(n-1)):
    ngram = ''
    for j in range(n):
        ngram +=  Chinese_train_string[i+j]
    create(ngram, ngram_training)

#Deal with the last a few characters 
n = n-1
while n!=0 :
    create(Chinese_train_string[-n:],ngram_training)
    n-=1

In [8]:
## define a function to find the counts in the tree structure. This function traverse the tree to find the counts of ngram  
def find_counts (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)                       ## find the counts of ngram
    if n == 1:  ## base case
        if ngram not in ngram_training:
            return 0
        else:
            return ngram_training[ngram][0]    
    else:
        if ngram[0] not in ngram_training:
            return 0
        else:
            return find_counts(ngram[1:],ngram_training[ngram[0]][1])

In [9]:
## define a function to find the subtree 
def find_subtree (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)
    if n == 1:
        if ngram not in ngram_training:
            temp ={}
            return temp
        else:
            return ngram_training[ngram][1]    
    else:
        if ngram[0] not in ngram_training:
            temp = {}
            return temp
        else:
            return find_subtree(ngram[1:],ngram_training[ngram[0]][1])

In [10]:
## Define the recursive function for prediction with witten-bell smoothing
## This function returns p(w/u)
def prediction_witten_bell_smoothing(n,w,position_predict,word_string_han): ## n means n-gram, w is the character to predict, 
                                                                        ## position_predict is the postion to be predicted                                                                       
    ## construct u according to the correct han characters     
    u=''
    for i in range(n-1,0,-1):
        u += word_string_han[position_predict-i]   
    
        
    if len(u)==0: ## base case, return p(w)  
        N = len(Chinese_train_string) # number of word token-not including <s>
        n_1_plus = len(word_count_unigram)## number of word type seen at least one time -not including <s>
        n_word_type = len(word_count_unigram)+1 ## number of word type_include unkown word
        lamda = N/(N+n_1_plus)
        if w in word_count_unigram:
            return lamda*ngram_training[w][0]/N + (1-lamda)/n_word_type   ## ngram_training[w][0] represents c(w), return p(w)
        else:
            return (1-lamda)/n_word_type
        
    if find_counts (u, ngram_training) == 0:
        return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)
    else:        
        ## calculate c(u_dot) and n1+(u_dot)        
        sub_tree_under_u = find_subtree (u, ngram_training)
        count_u_dot = 0
        for i in sub_tree_under_u:
            count_u_dot+=sub_tree_under_u[i][0]     
        
        count_1plus_u_dot = 0
        for i in sub_tree_under_u:
            count_1plus_u_dot+=1 
            
        if count_u_dot == 0:
            return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)
                
        ## calculate lamda_u
        lamda_u = count_u_dot/(count_u_dot+count_1plus_u_dot)
        
        ## calculate c(uw)
        c_uw = find_counts(u+w, ngram_training)
        
        return lamda_u*c_uw/count_u_dot+(1-lamda_u)*prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)

In [28]:
## Read in dev.pin as a big string, and convert to list with all the words 
Chinese_dev_pinyin_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/dev.pin',encoding='utf8') as Chinese_dev_pinyin:
    for line in Chinese_dev_pinyin:
        Chinese_dev_pinyin_string += line
word_dev_pin_list=re.findall(r'\S+|\n',Chinese_dev_pinyin_string)
#'\S+|\n' will match all the combinations of none whitespace character with length 1 or more (\S+) or new line (\n).
#word_dev_pin_list = Chinese_dev_pinyin_string.split()
#print (word_dev_pin_list)

In [22]:
print (len(word_dev_pin_list))
print (word_dev_pin_list)

1816
['p', 'i', 'g', 'g', 'y', 'b', 'o', 'x', ':', '<space>', 'ni', 'yong', 'm', 'a', 'c', 'de', 'l', 'a', 'u', 'n', 'c', 'h', 'd', 'bu', '\n', 'o', 'f', 'a', 'n', ':', '<space>', 'ou', 'er', 'yong', '\n', 'o', 'f', 'a', 'n', ',', '<space>', 'ni', 'zai', 'yong', 'shen', 'me', 'v', 'p', 's', 'ya', '？', '\n', 'p', 'i', 'g', 'g', 'y', 'b', 'o', 'x', ':', '<space>', 'guan', 'li', 'fu', 'wu', 'chao', 'fang', 'bian', '\n', 'o', 'f', 'a', 'n', ':', '<space>', 's', 'y', 's', 't', 'e', 'm', 'd', 'shi', 'xiao', 'fang', 'l', 'a', 'u', 'n', 'c', 'h', 'd', 'de', '？', '\n', 's', 'y', 's', 't', 'e', 'n', 'd', 'hai', 'zi', 'dai', 'd', 'b', 'u', 's', 'gong', 'neng', '，', 'u', 'd', 'e', 'v', '\n', 'ye', 'bu', 'yong', 'le', '\n', 'na', 'he', 'u', 'p', 's', 't', 'a', 'r', 't', 'bi', 'ne', '？', '\n', 'suo', 'you', 'gua', 'zai', 'dian', 'dou', 'shi', 'yi', 'ge', 's', 'y', 's', 't', 'e', 'm', 'd', '<space>', 't', 'a', 'r', 'g', 'e', 't', '\n', 'u', 'p', 's', 't', 'a', 'r', 't', '<space>', 'tai', 'lan', 'le',

In [26]:
## Read in dev.han as a big string, remove all the newline character 
Chinese_dev_han_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/dev.han',encoding='utf8') as Chinese_dev_han:
    for line in Chinese_dev_han:
        Chinese_dev_han_string += line
print (len(Chinese_dev_han_string))

1816


In [41]:
## Make prediction on the dev file
n=10
Chinese_prediction = {}
for position in range(n-1, len(word_dev_pin_list)):
    prediction_word = {}
    if word_dev_pin_list[position] == '<space>':
        Chinese_prediction[position] = " "
    else:
        if word_dev_pin_list[position] in Chinese_pinyin_character_map:
            for word in Chinese_pinyin_character_map[word_dev_pin_list[position]]: 
                prediction_word[word] = prediction_witten_bell_smoothing(n,word,position,Chinese_dev_han_string)
            Chinese_prediction[position]=[key for key, val in prediction_word.items() if val == max(prediction_word.values())][0]
        else: 
            Chinese_prediction[position] = word_dev_pin_list[position]
            
Correct_prediction_count = 0
Correct_position = []
for position in range(n-1, len(word_dev_pin_list)):
    if Chinese_prediction[position] == Chinese_dev_han_string[position]:
        Correct_prediction_count+=1
        Correct_position.append(position)

print (Correct_prediction_count/(len(Chinese_dev_han_string)-(n-1)))


0.7921503593145384


In [42]:
print(Chinese_prediction)

{7: 'x', 8: ':', 9: ' ', 10: '你', 11: '用', 12: '呣', 13: '啊', 14: 'c', 15: '的', 16: 'l', 17: '啊', 18: 'u', 19: '嗯', 20: 'c', 21: 'h', 22: 'd', 23: '不', 24: '\n', 25: '哦', 26: 'f', 27: '啊', 28: '嗯', 29: ':', 30: ' ', 31: '欧', 32: '尔', 33: '用', 34: '\n', 35: '哦', 36: 'f', 37: '啊', 38: '嗯', 39: ',', 40: ' ', 41: '你', 42: '在', 43: '用', 44: '什', 45: '么', 46: 'v', 47: 'p', 48: 's', 49: '呀', 50: '？', 51: '\n', 52: 'p', 53: 'i', 54: 'g', 55: 'g', 56: 'y', 57: 'b', 58: '哦', 59: 'x', 60: ':', 61: ' ', 62: '关', 63: '理', 64: '服', 65: '务', 66: '超', 67: '方', 68: '便', 69: '\n', 70: '哦', 71: 'f', 72: '啊', 73: '嗯', 74: ':', 75: ' ', 76: 's', 77: 'y', 78: 's', 79: 't', 80: '额', 81: '呣', 82: 'd', 83: '是', 84: '小', 85: '方', 86: 'l', 87: '啊', 88: 'u', 89: '嗯', 90: 'c', 91: 'h', 92: 'd', 93: '的', 94: '？', 95: '\n', 96: 's', 97: 'y', 98: 's', 99: 't', 100: '额', 101: '嗯', 102: 'd', 103: '还', 104: '自', 105: '带', 106: 'd', 107: 'b', 108: 'u', 109: 's', 110: '工', 111: '能', 112: '，', 113: 'u', 114: 'd', 115: '额', 

In [43]:
for i in Correct_position:
    print (Chinese_dev_han_string[i])

x
:
 
你
用
c
的
l
u
c
h
d
不


f
:
 
尔
用


f
,
 
你
在
用
什
v
p
s
呀
？


p
i
g
g
y
b
x
:
 
理
服
务
超
方
便


f
:
 
s
y
s
t
d
是
l
u
c
h
d
的
？


s
y
s
t
d
还
自
带
d
b
u
s
能
，
u
d
v


也
不
用
了


那
和
u
p
s
t
r
t
呢
？


所
有
载
点
都
是
一
个
s
y
s
t
d
 
t
r
g
t


u
p
s
t
r
t
 
太
烂
了
，
只
有
u
b
u
t
u
 
用


u
p
s
t
r
t
是
毛


l
u
c
h
d
的
设
计
非
常
好


s
y
s
t
d
就
的
l
u
c
h
d


哦
，
这
样
啊


c
下
添
加
一
个
服
务
就
写
几
行
p
l
i
s
t
就
行


f
,
 
都
c
了
在
用
l
i
u
x
？


f
:
 
只
能
自
己
编
译
t
w
i
k
l


一
l
g
，
前
r
c
.
d
启
动
败
不
会
显
l
g
，
s
y
s
t
d
还
带
分
析
能


c
l
_
:
 
用


f
,
 
不
浪
？


c
l
_
:
 
上
网
本
用


c
l
 
他
器
多


c
l
 
代


等
w
y
l
d
熟
了
就
给
c
装


u
g
b
j
g
d
,
 
.
.
.
.
.
.


f
,
 
你
真
.
.
.


c
l
_
:
 
服
务
器
上
得
用
l
i
u
x
啊


u
g
b
j
g
d
:
 
学
新
词
行
么


p
i
g
g
y
b
x
,
 
人
在
器
上
用
c
吧
.
.
.


c
l
_
:
 
哥
不


c
l
_
:
 
有
c
 
s
x
 
s
r
v
r
版
，
好
像
只
有
那
些
做
体
辑
的
用


腾
l
i
u
x
经
车
了


f
:
 
饭
饭
，
帮
忙
测
试
。
。


p
i
g
g
y
b
x
,
 
那
是
吃
了
的


f
:
 
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
/
l
v
i
r
x
g
/
d
u
b
_
t
i
z


l
v
i
_
r
x
g
 
⇪
 


In [37]:
for i in Correct_position:
    print (word_dev_pin_list[i])

x
:
<space>
c
l
u
c
h
d


f
:
<space>


f
,
<space>
v
p
s
？


p
i
g
g
y
b
x
:
<space>


f
:
<space>
s
y
s
t
d
l
u
c
h
d
？


s
y
s
t
d
d
b
u
s
，
u
d
v




u
p
s
t
r
t
？


s
y
s
t
d
<space>
t
r
g
t


u
p
s
t
r
t
<space>
，
u
b
u
t
u
<space>


u
p
s
t
r
t


l
u
c
h
d


s
y
s
t
d
l
u
c
h
d


，


c
p
l
i
s
t


f
,
<space>
c
l
i
u
x
？


f
:
<space>
t
w
i
k
l


l
g
，
r
c
.
d
l
g
，
s
y
s
t
d


c
l
_
:
<space>


f
,
<space>
？


c
l
_
:
<space>


c
l
<space>


c
l
<space>


w
y
l
d
c


u
g
b
j
g
d
,
<space>
.
.
.
.
.
.


f
,
<space>
.
.
.


c
l
_
:
<space>
l
i
u
x


u
g
b
j
g
d
:
<space>


p
i
g
g
y
b
x
,
<space>
c
.
.
.


c
l
_
:
<space>


c
l
_
:
<space>
c
<space>
s
x
<space>
s
r
v
r
，


l
i
u
x


f
:
<space>
，
。
。


p
i
g
g
y
b
x
,
<space>


f
:
<space>
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
/
l
v
i
r
x
g
/
d
u
b
_
t
i
z


l
v
i
_
r
x
g
<space>
⇪
<space>
t
i
:
<space>
l
v
i
r
x
g
/
d
u
b
_
t
i
z
<space>
·
<space>
G
i
t
H
u
b


<space>
=
>
<space>
.
.
.




l
v
i
_
r
x
g
:
<space>


u
g
b
j
g
d
,
<spa

In [35]:
print(word_dev_pin_list[7])
print(Chinese_dev_han_string[7])
print(Chinese_dev_han_string)

x
x
piggybox: 你用mac的launchd不
ofan: 偶尔用
ofan, 你在用什麼vps呀？
piggybox: 管理服务超方便
ofan: systemd是效仿launchd的？
systend还自带dbus功能，udev
也不用了
那和upstart比呢？
所有挂载点都是一个systemd target
upstart 太烂了，只有ubuntu 用
upstart是毛
launchd的设计非常好
systemd就抄的launchd
哦，这样啊
mac下添加一个服务就写几行plist就行
ofan, 都mac了還在用linux？
ofan: 只能自己编译twinkle
统一log，之前rc.d启动失败不会显示log，systemd还带分析功能
cleamoon_: 用
ofan, 不感覺浪費嗎？
cleamoon_: 上网本用
cleamoon 他机器多
cleamoon 二代
等wayland成熟了就给mac装
mugebjgd, 的確......
ofan, 你真閑...
cleamoon_: 服务器上还得用linux啊
mugebjgd: 学点新台词行么
piggybox, 沒人在服務器上用mac吧...
cleamoon_: 哥不闲
cleamoon_: 有mac osx server版，好像只有那些做媒体剪辑的用
折腾linux已经轻车熟路了
ofan: 饭饭，帮忙测试。。
piggybox, 那是吃飽了撐的
ofan: https://github.com/alvinrxg/douban_timezone
alvin_rxg ⇪ ti: alvinrxg/douban_timezone · GitHub
把本子的亮度調到最大 => 晃瞎哥的狗眼...
睡觉
alvin_rxg: 得出门了
mugebjgd, 好早
>_< 有空帮我测吧。
cleamoon 11点
cleamoon 还早？
mugebjgd, 很早
mugebjgd, 2點睡才正常
cleamoon_: 和你这样的学生没法比啊
cleamoon_: 你是不用7点半起床
mugebjgd, 上班才舒服吧，晚點去也無所謂
mugebjgd, 我6點起床...
cleamoon_: 周6飞米国
mugebjgd, 在地鐵上睡1小時
cleamoon_: 要多在家陪陪老婆
mu

In [44]:
## Read in test.pin as a big string, and convert to list with all the words 
Chinese_test_pinyin_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/test.pin',encoding='utf8') as Chinese_test_pinyin:
    for line in Chinese_test_pinyin:
        Chinese_test_pinyin_string += line
        
word_test_pin_list = re.findall(r'\S+|\n',Chinese_test_pinyin_string)
print (len(word_test_pin_list))

1646


In [45]:
## Read in dev.han as a big string, remove all the newline character 
Chinese_test_han_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/test.han',encoding='utf8') as Chinese_test_han:
    for line in Chinese_test_han:
        Chinese_test_han_string += line
        
print (len(Chinese_test_han_string))

1646


In [54]:
## Make prediction on the test file
n=10
English_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                   'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

Chinese_prediction = {}
for position in range(n-1, len(word_test_pin_list)):
    prediction_word = {}
    if word_test_pin_list[position] == '<space>':
        Chinese_prediction[position] = " "
        
    elif word_test_pin_list[position] in English_letters:
        Chinese_prediction[position] = word_test_pin_list[position]
        
                
    else:        
        
        if word_test_pin_list[position] in Chinese_pinyin_character_map:
            for word in Chinese_pinyin_character_map[word_test_pin_list[position]]: 
                prediction_word[word] = prediction_witten_bell_smoothing(n,word,position,Chinese_test_han_string)
            Chinese_prediction[position]=[key for key, val in prediction_word.items() if val == max(prediction_word.values())][0]
        else:
            Chinese_prediction[position]=word_test_pin_list[position]
            
Correct_prediction_count = 0
for position in range(n-1, len(word_test_pin_list)):
    if Chinese_prediction[position] == Chinese_test_han_string[position]:
        Correct_prediction_count+=1

print (Correct_prediction_count/(len(Chinese_test_han_string)-(n-1)))

0.8821014050091631


In [51]:
English_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                   'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [52]:
print(English_letters)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [55]:
len('\n')

1